In [1]:
import datetime
import pandas as pd
%matplotlib inline

In [2]:
def groupby_min(df, dur):
    temp = df.copy()
    temp.index = map(lambda x: pd.datetime.combine(datetime.datetime.today(), x), temp.index.time)
    return temp.groupby("cam_id").resample(dur).mean()

# Load data, and only use weekdays.
df = pd.read_csv("../data/external/hadive-data.csv",
                 usecols=["cam_id", "date", "cam_dir", "count", "imtime"],
                 dtype={"cam_id": int, "date": str, "cam_dir": str, "count": int, "imtime": str},
                 parse_dates=["date"], infer_datetime_format=True, index_col="date")
df = df[df.index.weekday < 5]

In [3]:
dfs = []
for cam_id in df.cam_id.unique():
    dfs.append(groupby_min(df[df["cam_id"] == cam_id], "5Min").set_index("cam_id", append=True))

In [4]:
df_ = pd.concat(dfs)
df_ = df_.reset_index(level=2, drop=True).reset_index().fillna(0)
df_.columns = ["cam", "date", "count"]
df_.set_index("cam").to_csv("..data/external/weekdayavg.csv")